# Question 1  : J'ai choisi de faire la 2ème méthode 

## Work With mps for MacOS M1 Pro and Max 
also with cuda but not tested

In [246]:
import platform
platform.platform()

'macOS-13.1-arm64-arm-64bit'

In [247]:
# import the libraries

!python3 --version
!pip3 install torch
!python3.11 -m pip install --upgrade pip


import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

Python 3.9.6


### Check if MPS is installed

In [248]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    gpu = torch.device("mps")
    x = torch.ones(5, device=gpu)
    mps_active = True
    print (x)

tensor([1., 1., 1., 1., 1.], device='mps:0')


### Download data + encode

In [249]:
# check if the gpu is available

train_on_gpu = torch.cuda.is_available()

# check if mps acceleration is available

train_with_mps = torch.has_mps

# import the data
# the data is a text file with a lot of words of the english language
# download the data from https://www.gutenberg.org/files/11/11-0.txt

!wget https://www.gutenberg.org/files/11/11-0.txt

!wget https://www.gutenberg.org/files/66927/66927-0.txt

!wget https://www.gutenberg.org/cache/epub/18092/pg18092.txt


# read the data

with open('pg18092.txt', 'r') as f:
    text = f.read()

# print the first 1000 characters of the data

print(text[:1000])

print("test")
# create a dictionary of the characters in the data

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
# encode the text

encoded = np.array([char2int[ch] for ch in text])



# print the first 1000 characters of the encoded data

print(encoded[:1000])


--2023-04-18 19:55:25--  https://www.gutenberg.org/files/11/11-0.txt
Résolution de www.gutenberg.org (www.gutenberg.org)… 2610:28:3090:3000:0:bad:cafe:47, 152.19.134.47
Connexion à www.gutenberg.org (www.gutenberg.org)|2610:28:3090:3000:0:bad:cafe:47|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 174313 (170K) [text/plain]
Sauvegarde en : « 11-0.txt »

11-0.txt            100%[===================>] 170,23K   680KB/s    ds 0,3s    

2023-04-18 19:55:27 (680 KB/s) — « 11-0.txt » sauvegardé [174313/174313]

--2023-04-18 19:55:27--  https://www.gutenberg.org/files/66927/66927-0.txt
Résolution de www.gutenberg.org (www.gutenberg.org)… 2610:28:3090:3000:0:bad:cafe:47, 152.19.134.47
Connexion à www.gutenberg.org (www.gutenberg.org)|2610:28:3090:3000:0:bad:cafe:47|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 532433 (520K) [text/plain]
Sauvegarde en : « 66927-0.txt »

66927-0.txt         100%[===================>] 519,95K  

### Create batch 

In [250]:
# define the function to get the batches of the data

def get_batches(arr, n_seqs, n_steps):

    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size

    arr = arr[:batch_size * n_batches]
    arr = arr.reshape((n_seqs, -1))

    for n in range(0, arr.shape[1], n_steps):
        x = arr[:, n:n+n_steps]
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y



### Creation of the model 

In [318]:
# define the class to create the model

class CharRNN(nn.Module):

    def __init__(self, tokens, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden
    
    # define function "to" to send to gpu or mps device if available

    def to(self, *args, **kwargs):
        super().to(*args, **kwargs)
        return self
    
     
    



### One-Hot encoder

In [319]:
# define the function to one hot encode the data

def one_hot_encode(arr, n_labels):

    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot



### define the train with verification of cpu, mps or cuda run

In [320]:
# define the function to train the model

def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):

    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(train_on_gpu):
        net.cuda()

    if (torch.backends.mps.is_available()):
        net.to(gpu)
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            h = tuple([each.data for each in h])

            if (torch.backends.mps.is_available()):
                inputs = inputs.to(gpu)
                targets = targets.to(gpu)
                h = tuple([each.data.to(gpu) for each in h])

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
                h = tuple([each.data.cuda() for each in h])
            

            net.zero_grad()

            output, h = net(inputs, h)

            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            if counter % print_every == 0:
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y

                    if (torch.backends.mps.is_available()):
                        inputs = inputs.to(gpu)
                        targets = targets.to(gpu)

                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())

                    val_losses.append(val_loss.item())

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))



In [321]:
# define the function to predict the next character

def predict(net, char, h=None, top_k=None):

    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)


    h = tuple([each.data for each in h])

    if (torch.backends.mps.is_available()):
        inputs = inputs.to(gpu)
        h = tuple([each.to(gpu) for each in h])

    if(train_on_gpu):
        inputs = inputs.cuda()
        h = tuple([each.cuda() for each in h])
    
    out, h = net(inputs, h)

    p = F.softmax(out, dim=1).data

    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.cpu().numpy().squeeze()

    p = p.cpu().numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())

    return net.int2char[char], h





In [322]:
# launch the model training

n_hidden=1024
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)


net.to(gpu)


print(net)

batch_size = 128
seq_length = 100
n_epochs = 20
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)



CharRNN(
  (lstm): LSTM(104, 1024, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1024, out_features=104, bias=True)
)
Epoch: 1/20... Step: 10... Loss: 3.2122... Val Loss: 3.3380
Epoch: 1/20... Step: 20... Loss: 3.1260... Val Loss: 3.3408
Epoch: 2/20... Step: 30... Loss: 3.1565... Val Loss: 3.3262
Epoch: 2/20... Step: 40... Loss: 3.1237... Val Loss: 3.3208
Epoch: 2/20... Step: 50... Loss: 3.1118... Val Loss: 3.3167
Epoch: 3/20... Step: 60... Loss: 3.0887... Val Loss: 3.3050
Epoch: 3/20... Step: 70... Loss: 3.0779... Val Loss: 3.2695
Epoch: 3/20... Step: 80... Loss: 3.0329... Val Loss: 3.2563
Epoch: 4/20... Step: 90... Loss: 2.9155... Val Loss: 3.1979
Epoch: 4/20... Step: 100... Loss: 2.8494... Val Loss: 3.1675
Epoch: 4/20... Step: 110... Loss: 2.7539... Val Loss: 3.1061
Epoch: 5/20... Step: 120... Loss: 2.7069... Val Loss: 3.0564
Epoch: 5/20... Step: 130... Loss: 2.5734... Val Loss: 2.9710
Epoch: 5/20... Step: 140... L

In [444]:
# define the function to generate the text

def sample(net, size, prime='L', top_k=None):

    x = np.array([[net.char2int[ch] for ch in prime]])
    h = net.init_hidden(1)

    if(train_on_gpu):
        net.cuda()
    if (torch.backends.mps.is_available()):
        net.to(gpu)
    else:
        net.cpu()

    chars = [ch for ch in prime]
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)




    
# generate the text

a = sample(net, 3, prime='appartem', top_k=5)
print(a)

# save the model

model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
                'n_layers': net.n_layers,
                'state_dict': net.state_dict(),
                'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

# load the model

with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

# print the generated text

b= sample(loaded, 1, top_k=5, prime="docteu")
print(b)


appartement
docteur
1


In [446]:
#calculate the perplexity

def perplexity(net, data, seq_length=100):

    net.eval()

    n_chars = len(net.chars)
    criterion = nn.CrossEntropyLoss()

    h = net.init_hidden(1)
    val_losses = []
    for x, y in get_batches(data, 1, seq_length):
        x = one_hot_encode(x, n_chars)
        x, y = torch.from_numpy(x), torch.from_numpy(y)

        h = tuple([each.data for each in h])

        inputs, targets = x, y
        if(train_on_gpu):
            inputs, targets = inputs.cuda(), targets.cuda()

        if (torch.backends.mps.is_available()):
            inputs = inputs.to(gpu)
            targets = targets.to(gpu)
            

        output, h = net(inputs, h)
        val_loss = criterion(output, targets.view(1*seq_length).long())

        val_losses.append(val_loss.item())

    net.train()

    return np.exp(np.mean(val_losses))

print(perplexity(loaded, encoded, seq_length=100))


4.349862745999265


# Question 2

## Comment pourriez-vous utiliser TF-IDF également pour simplifier ou améliorer vos modèles?

Pour la première methode je pense que TF-IDF pourrait reduire le nombre de caractères de la fenêtre d'entrée car avec les poids pour chaque caractère calculer à partir de TF-IDF, on peut identifier les caractère qui ont le plus de poids pour les caractère suivants.
On reduit la dimensionnalité en se concentrant sur ces caractère uniquement.  

Pour la deuxième, je pense que ça ne peut pas améliorer le modèle enormément mais on pourrait améliorer la perplexité. 
on pourrait utiliser TF-IDF pour filtrer les caractères peu fréquents de la même manière que pour la premère méthode 